In [ ]:
import os
from dotenv import find_dotenv, load_dotenv
from shapely.ops import orient

load_dotenv(find_dotenv())
src_path = os.environ.get("PROJECT_SRC")
os.chdir(src_path)

src_path

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")
import generators
from data_reader_writer import data_writer

# Costing Analysis of 2030 Generation

In [ ]:
year = 2030
FES_scenario = "Leading The Way"
FES_year = 2022
euro_to_gbp = 1.20

In [ ]:
PV_investment: dict[str, float] = {
    "PV module": 0.11 * 10 ** 6,  # euro/MW
    "Inverter": 0.02 * 10 ** 6,  # euro/MW
    "Transformer": 0.04 * 10 ** 6,  # euro/MW
    "Installation": 0.08 * 10 ** 6,  # euro/MW
    "Soft costs": 0.02 * 10 ** 6,  # euro/MW
    "Residual balance of plant, mark-up & contingency cost": 0.03 * 10 ** 6,  # euro/MW
}

ccgt_investment: dict[str, float] = {
    "Equipment nominal investment": 0.64 * 10 ** 6,  # euro/MW
    "Installation nominal investment": 0.19 * 10 ** 6,  # euro/MW
    "Fixed O&M per year": 27_800,  # euro/MW
}

wind_offshore_investment: dict[str, float] = {
    "Fixed bottom cost per year": 39_000,  # euro/MW
}

wind_onshore_investment: dict[str, float] = {
    "Equipment nominal investment": 0.978 * 10 ** 6,  # euro/MW
    "Installation nominal investment": 0.101 * 10 ** 6,  # euro/MW
    "Grid connection nominal investment": 0.019 * 10 ** 6,  # euro/MW
    "Land purchase nominal investment": 0.122 * 10 ** 6,  # euro/MW
    "Decommissioning existing turbines": 0.035 * 10 ** 6,  # euro/MW
    "Purchase of neighbour settlements": 0.077 * 10 ** 6,  # euro/MW
    "Fixed O&M per year": 19_676,  # euro/MW
}

biomass_investment: dict[str, float] = {
    "Equipment nominal investment": 6.00 * 10 ** 6,  # euro/MW
    "Installation nominal investment": 3.70 * 10 ** 6,  # euro/MW
    "Fixed O&M per year": 273_000,  # euro/MW
}

nuclear_investment: dict[str, float] = {
    ""
}

## FES Data

In [ ]:
start = str(year) + "-01-01 00:00:00"
end = str(year) + "-12-31 03:30:00"
# time step as fraction of hour
time_step = 0.5
if year > 2020:
    data_writer(start, end, time_step, year, demand_dataset="eload", year_baseline=2012, scenario=FES_scenario,
                FES=FES_year)
if year <= 2020:
    data_writer(start, end, time_step, year, demand_dataset="historical")

df_generators_2030 = pd.read_csv("LOPF_data/generators.csv", index_col=0)

# get 2025 data
data_writer("2025-01-01 00:00:00", "2025-12-31 23:30:00", time_step, "2025", demand_dataset="eload", year_baseline=2012,
            scenario=FES_scenario, FES=FES_year)
df_generators_2025 = pd.read_csv("LOPF_data/generators.csv", index_col=0)

In [ ]:
generators_p_nom_2030 = df_generators_2030.p_nom.groupby(df_generators_2030.carrier).sum().sort_values()
if year > 2020:
    generators_p_nom_2030.drop("Unmet Load", inplace=True)
generators_p_nom_2030.drop(generators_p_nom_2030[generators_p_nom_2030 < 50].index, inplace=True)

generators_p_nom_2025 = df_generators_2025.p_nom.groupby(df_generators_2025.carrier).sum().sort_values()
if year > 2020:
    generators_p_nom_2025.drop("Unmet Load", inplace=True)
generators_p_nom_2025.drop(generators_p_nom_2025[generators_p_nom_2025 < 50].index, inplace=True)

generators_p_nom_FES = generators_p_nom_2030 - generators_p_nom_2025

plt.rcParams.update({"font.size": 22})
# bar chart
plt.figure(figsize=(15, 10))
plt.bar(generators_p_nom_FES.index, generators_p_nom_FES.values / 1000)
plt.xticks(generators_p_nom_FES.index, rotation=90)
plt.ylabel("GW")
plt.grid(color="grey", linewidth=1, axis="both", alpha=0.5)
plt.title("Installed capacity in year " + str(year))
plt.show()

In [ ]:
generators_p_nom_2030.index

In [ ]:
solar_cost: float = 0.00
offshore_cost: float = 0.00
onshore_cost: float = 0.00
biomass_cost: float = 0.00
ccgt_cost: float = 0.00

for gen_tech in generators_p_nom_2030.index:
    if gen_tech == "Solar Photovoltaics":
        for key, value in PV_investment.items():
            solar_cost += value * generators_p_nom_2030["Solar Photovoltaics"]

    if gen_tech == "Wind Offshore":
        for key, value in wind_offshore_investment.items():
            offshore_cost += value * generators_p_nom_2030["Wind Offshore"]

    if gen_tech == "Wind Onshore":
        for key, value in wind_onshore_investment.items():
            onshore_cost += value * generators_p_nom_2030["Wind Onshore"]

    if gen_tech == "Biomass (dedicated)":
        for key, value in biomass_investment.items():
            biomass_cost += value * generators_p_nom_2030["Biomass (dedicated)"]

    if gen_tech == "Natural Gas":
        for key, value in ccgt_investment.items():
            ccgt_cost += value * generators_p_nom_2030["Natural Gas"]

solar_cost *= 1.20
offshore_cost *= 1.20
onshore_cost *= 1.20
biomass_cost *= 1.20
ccgt_cost *= 1.20

# print(
#     f"{"Solar":<15}{"|":^1}{u"\N{POUND SIGN}":>2}{solar_cost/10**6:>9.2f} {"M":<1} \n"
#     f"{"Wind Offshore":<15}{"|":^1}{u"\N{POUND SIGN}":>2}{offshore_cost/10**6:>9.2f} {"M":<1} \n"
#     f"{"Wind Onshore":<15}{"|":^1}{u"\N{POUND SIGN}":>2}{onshore_cost/10**6:>9.2f} {"M":<1} \n"
#     f"{"Biomass":<15}{"|":^1}{u"\N{POUND SIGN}":>2}{biomass_cost/10**6:>9.2f} {"M":<1} \n"
#     f"{"CCGT":<15}{"|":^1}{u"\N{POUND SIGN}":>2}{ccgt_cost/10**6:>9.2f} {"M":<1} \n"
# )

## Neso Data Further Flex

In [ ]:
gen_data: dict[str, float] = {
    "Solar Photovoltaics": 47.4 * 10 ** 3,  # MW
    "Wind Offshore": 50.6 * 10 ** 3,  # MW
    "Wind Onshore": 27.3 * 10 ** 3,  # MW
    "Biomass (dedicated)": 4.0 * 10 ** 3,  # MW
    "Natural Gas": 35.0 * 10 ** 3,  # MW
    "Nuclear": 3.5 * 10 ** 3,  # MW
    "Hydrogen": 0.3 * 10 ** 3,  # MW
}

df_neso_ff = pd.DataFrame(list(gen_data.items()), columns=["Technology", "Capacity [MW]"].set_index("Technology"))
df_neso_ff